# SQL + OpenAI

In this Jupyter notebook, we test querying a SQL database using OpenAI API calls. We use the Chinook database, and determine how to call the AI to respond with natural language.

First, we can install the required dependencies.

In [3]:
%%writefile requirements.txt

langchain==0.2.8
langchain_community==0.2.7
langchain_core==0.2.19
langchain_openai==0.1.16

Overwriting requirements.txt


In [ ]:
pip install -r requirements.txt

Now, let's get the OPENAI_API_KEY.

In [13]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

Here, we test how to work with the SQL Database in Python. First, we create the Chinook.db using the guide from [here](https://database.guide/2-sample-databases-sqlite/).

In [5]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

Now, we can use Langchain chains to ask the AI to query the SQL tables.

In [14]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

Now that we know that it works, we can use prompt templates along with the chains in order to get a proper natural language response.

In [15]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user's question.
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are a total of 8 employees.'